<a href="https://colab.research.google.com/github/minzix/garbage_sorting/blob/main/VGG16model_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
!ls
!ls "/gdrive/My Drive/ColabNotebooks/"

sample_data
 dataset
 DenseNet.ipynb
'DL23-Lab3-pytorch_nn_autograd.ipynb의 사본'
 EfficientNet.ipynb
 numpy_inflearn.ipynb
'quickstart_tutorial.ipynb의 사본'
'SMWU-2023-DL-Lab1-pytorch101.ipynb의 사본'
'SMWU-2023-DL-Lab2-FC_class.ipynb의 사본'
 Untitled0.ipynb
 VGG16model.ipynb


In [ ]:
import os
import numpy as np
from PIL import Image

# 데이터셋 디렉토리 경로
dataset_dir = '/gdrive/My Drive/ColabNotebooks/dataset'

# 이미지 전처리 함수
def preprocess_image(image_path):
    # 이미지 불러오기
    image = Image.open(image_path)

    # 이미지 크기 조정
    image = image.resize((224, 224))  # 원하는 크기로 조정

    # 이미지를 numpy 배열로 변환
    image_array = np.array(image)

    # 전처리된 이미지 반환
    return image_array

# 클래스 레이블과 해당 폴더의 이미지 경로 가져오기
class_labels = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
image_paths = []

for label in class_labels:
    folder_path = os.path.join(dataset_dir, 'train', label)
    images = os.listdir(folder_path)
    images = [os.path.join(folder_path, image) for image in images]
    image_paths.extend(images)

# 이미지 데이터 전처리
preprocessed_images = []

for image_path in image_paths:
    preprocessed_image = preprocess_image(image_path)
    preprocessed_images.append(preprocessed_image)

# 전처리된 이미지 데이터를 사용하여 추가적인 처리를 수행할 수 있습니다.
# 예를 들어, 전처리된 이미지 데이터를 NumPy 배열로 변환하여 모델에 입력으로 사용할 수 있습니다.
preprocessed_images = np.array(preprocessed_images)

# 전처리된 이미지 데이터의 크기 확인
print('전처리된 이미지 데이터 크기:', preprocessed_images.shape)



전처리된 이미지 데이터 크기: (1515, 224, 224, 3)


In [ ]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# 데이터셋 디렉토리 경로
dataset_dir = '/gdrive/My Drive/ColabNotebooks/dataset'

# 이미지 전처리 함수
def preprocess_image(image_path):
    # 이미지 불러오기
    image = Image.open(image_path)

    # 이미지 크기 조정
    image = image.resize((224, 224))
    image_array = np.array(image)
    return image_array

# 클래스 레이블과 해당 폴더의 이미지 경로 가져오기
class_labels = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
image_paths = []
labels = []

for label_idx, label in enumerate(class_labels):
    folder_path = os.path.join(dataset_dir, 'train', label)
    images = os.listdir(folder_path)
    images = [os.path.join(folder_path, image) for image in images]
    image_paths.extend(images)
    labels.extend([label_idx] * len(images))

# 이미지 데이터 전처리
preprocessed_images = []

for image_path in image_paths:
    preprocessed_image = preprocess_image(image_path)
    preprocessed_images.append(preprocessed_image)

# 전처리된 이미지 데이터를 NumPy 배열로 변환
preprocessed_images = np.array(preprocessed_images)

# 레이블을 One-Hot 인코딩
labels = to_categorical(labels)

# train-validation-test 분할
train_images, test_images, train_labels, test_labels = train_test_split(preprocessed_images, labels, test_size=0.2, random_state=42)
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.25, random_state=42)  # train 데이터의 0.25를 validation 데이터로 사용

# VGG16 모델 불러오기
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# VGG16 모델의 출력 위에 추가적인 레이어 쌓기
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(6, activation='softmax'))

# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 학습
model.fit(train_images, train_labels, validation_data=(val_images, val_labels), epochs=10, batch_size=32)

# 테스트 이미지 경로
test_image_path = '/gdrive/My Drive/ColabNotebooks/dataset/test'


58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/10
29/29 [==============================] - 2257s 78s/step - loss: 36.1922 - accuracy: 0.1848 - val_loss: 1.8221 - val_accuracy: 0.2409
Epoch 2/10
29/29 [==============================] - 2257s 78s/step - loss: 1.9529 - accuracy: 0.2354 - val_loss: 1.6978 - val_accuracy: 0.2013
Epoch 3/10
29/29 [==============================] - 2188s 76s/step - loss: 1.7115 - accuracy: 0.2530 - val_loss: 1.7244 - val_accuracy: 0.2211
Epoch 4/10
29/29 [==============================] - 2167s 75s/step - loss: 1.6476 - accuracy: 0.2992 - val_loss: 1.6129 - val_accuracy: 0.2970
Epoch 5/10
29/29 [==============================] - 2160s 75s/step - loss: 1.6160 - accuracy: 0.3190 - val_loss: 1.6189 - val_accuracy: 0.3267
Epoch 6/10
26/29 [=========================>....] - ETA: 3:25 - loss: 1.6054 - accuracy: 0.3065